In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, RandomTranslation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
%matplotlib inline

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)
train_path = '/content/drive/MyDrive/ML Datasets/cats_and_dogs_filtered/train'
val_path = '/content/drive/MyDrive/ML Datasets/cats_and_dogs_filtered/validation'

train_dataset = image_dataset_from_directory(train_path,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             seed=42)

test_dataset = image_dataset_from_directory(val_path,
                                            shuffle=True,
                                            batch_size=BATCH_SIZE,
                                            image_size=IMG_SIZE,
                                            seed=42)

In [ ]:
validation_dataset = np.array(train_test_split(train_dataset, test_size=0.2, random_state=42))
print(validation_dataset.shape)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
def augmentation():
  data_aug = tf.keras.Sequential()
  data_aug.add(RandomFlip('horizontal'))
  data_aug.add(RandomRotation(0.2))
  data_aug.add(RandomZoom((-0.2),(-0.2)))
  data_aug.add(RandomTranslation((0.2),(0.2)))

  return data_aug

data_augment = augmentation()

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
def cats_dogs_model(img_size):

  input_shape = img_size + (3,)

  base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                 include_top=False,
                                                 weights="imagenet")

  base_model.trainable = False

  inputs = tf.keras.Input(shape=input_shape)

  x = data_augment(inputs)
  x = preprocess_input(x)
  x = base_model(x, training=False)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dropout(0.2)(x)

  pred_layer = tf.keras.layers.Dense(1)

  outputs = pred_layer(x)
  model = tf.keras.Model(inputs, outputs)

  return model

In [ ]:
model = cats_dogs_model(IMG_SIZE)

In [ ]:
model.compile(optimizer=Adam(0.01),
              loss=BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
initial_epochs = 5
model.fit(train_dataset, batch_size=32, epochs=initial_epochs)

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print("Test accuracy: ", accuracy)
print("Loss: ", loss)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SIZE + (3,),
                                                 include_top=False,
                                                 weights="imagenet")

base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))
fine_tune_at = 120

for layer in base_model.layers[:fine_tune_at:]:
  layer.trainable = False

loss = tf.keras.losses.BinaryCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam(0.001)
metrics = ['accuracy']

model.compile(loss=loss, optimizer = optimizer, metrics=metrics)

In [ ]:
fine_tune_epochs = 2 * initial_epochs
fine_tuned = model.fit(train_dataset, epochs=fine_tune_epochs)

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print("Test accuracy: ", accuracy)
print("Loss: ", loss)